# Customizing embeddings

This notebook demonstrates one way to customize OpenAI embeddings to a particular task.

The input is training data in the form of [text_1, text_2, label] where label is +1 if the pairs are similar and -1 if the pairs are dissimilar.

The output is a matrix that you can use to multiply your embeddings. The product of this multiplication is a 'custom embedding' that will better emphasize aspects of the text relevant to your use case. In binary classification use cases, we've seen error rates drop by as much as 50%.

In the following example, I use 1,000 sentence pairs picked from the SNLI corpus. Each pair of sentences are logically entailed (i.e., one implies the other). These pairs are our positives (label = 1). We generate synthetic negatives by combining sentences from different pairs, which are presumed to not be logically entailed (label = -1).

For a clustering use case, you can generate positives by creating pairs from texts in the same clusters and generate negatives by creating pairs from sentences in different clusters.

With other data sets, we have seen decent improvement with as little as ~100 training examples. Of course, performance will be better with  more examples.

# 0. Imports

In [ ]:
# imports
from typing import List, Tuple # for type hints
from typing_extensions import Annotated 

import numpy as np  # for manipulating arrays
import pandas as pd  # for manipulating data in dataframes
import pickle  # for saving the embeddings cache
import plotly.express as px  # for plots
import random  # for generating run IDs
from sklearn.model_selection import train_test_split  # for splitting train & test data
import torch  # for matrix optimization

from zenml import step, pipeline

from zenml.logger import get_logger

logger = get_logger(__name__)

## 1. Inputs

Most inputs are here. The key things to change are where to load your datset from, where to save a cache of embeddings to, and which embedding engine you want to use.

Depending on how your data is formatted, you'll want to rewrite the process_input_data function.

In [ ]:
import requests
import zipfile
import os
import pandas as pd 
from typing_extensions import Tuple 

@step
def download_data(
    snli_url: str = 'https://nlp.stanford.edu/projects/snli/snli_1.0.zip',
    save_path: str = 'data/snli_1.0.zip',
    extracted_dir: str = 'data/snli_dataset'
) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """Returns the path to the downloaded dataset."""
    # Ensure the directory exists
    os.makedirs(os.path.dirname(save_path), exist_ok=True)

    # Check if the zip file has already been downloaded
    if not os.path.exists(save_path):
        # Download the file
        response = requests.get(snli_url, stream=True)

        # Check if the request was successful
        if response.status_code == 200:
            with open(save_path, 'wb') as f:
                f.write(response.content)
            print(f'Download complete. Dataset saved to {save_path}')
        else:
            print(f'Failed to download the dataset. Status code: {response.status_code}')
            return None  # Return None or raise an exception if the download failed

    # Unzip the downloaded file
    with zipfile.ZipFile(save_path, 'r') as zip_ref:
        zip_ref.extractall(extracted_dir)
        print(f'Extraction complete. Dataset extracted to {extracted_dir}')

    
    # Read the JSON Lines file into a DataFrame
    train_df = pd.read_json(os.path.join(extracted_dir, "snli_1.0/snli_1.0_train.jsonl"), lines=True)
    test_df = pd.read_json(os.path.join(extracted_dir, "snli_1.0/snli_1.0_test.jsonl"), lines=True)
    dev_df = pd.read_json(os.path.join(extracted_dir, "snli_1.0/snli_1.0_dev.jsonl"), lines=True)

    # Return the dataframes
    return train_df, test_df, dev_df

In [ ]:
download_data.entrypoint()

In [ ]:
@step
def process_input_data(df: pd.DataFrame, dataset_type: str, num_pairs_to_embed: int = 1000) -> pd.DataFrame:
    # you can customize this to preprocess your own dataset
    # output should be a dataframe with 3 columns: text_1, text_2, label (1 for similar, -1 for dissimilar)
    df["label"] = df["gold_label"]
    df = df[df["label"].isin(["entailment"])]
    df["label"] = df["label"].apply(lambda x: {"entailment": 1, "contradiction": -1}[x])
    df = df.rename(columns={"sentence1": "text_1", "sentence2": "text_2"})
    df = df[["text_1", "text_2", "label"]]
    df = df.head(num_pairs_to_embed)
    
    df.loc[:, "dataset"] = dataset_type
    
    return df


## 2. Load and process input data

In [ ]:
@pipeline
def load_and_process_data():
    train_df, test_df, dev_df = download_data()
    processed_train_df = process_input_data(train_df, dataset_type="train")
    processed_test_df = process_input_data(test_df, dataset_type="test")
    processed_dev_df = process_input_data(dev_df, dataset_type="dev")
    
load_and_process_data()

In [ ]:
# load data
from zenml.client import Client 

client = Client()

latest_run = client.get_pipeline("load_and_process_data").runs[-1]
df = latest_run.steps["process_input_data"].output.load()
df

## 3. Generate synthetic negatives

This is another piece of the code that you will need to modify to match your use case.

If you have data with positives and negatives, you can skip this section.

If you have data with only positives, you can mostly keep it as is, where it generates negatives only.

If you have multiclass data, you will want to generate both positives and negatives. The positives can be pairs of text that share labels, and the negatives can be pairs of text that do not share labels.

The final output should be a dataframe with text pairs, where each pair is labeled -1 or 1.

In [ ]:
# generate negatives
@step
def dataframe_of_negatives(dataframe_of_positives: pd.DataFrame, dataset_type: str) -> pd.DataFrame:
    """Return dataframe of negative pairs made by combining elements of positive pairs."""
    texts = set(dataframe_of_positives["text_1"].values) | set(
        dataframe_of_positives["text_2"].values
    )
    all_pairs = {(t1, t2) for t1 in texts for t2 in texts if t1 < t2}
    positive_pairs = set(
        tuple(text_pair)
        for text_pair in dataframe_of_positives[["text_1", "text_2"]].values
    )
    negative_pairs = all_pairs - positive_pairs
    df_of_negatives = pd.DataFrame(list(negative_pairs), columns=["text_1", "text_2"])
    df_of_negatives["label"] = -1
    
    df_of_negatives["dataset"] = dataset_type
    
    return df_of_negatives

In [ ]:
@step 
def concatenate_dataframes(
        train_df: pd.DataFrame,
        train_df_negatives: pd.DataFrame, 
        test_df: pd.DataFrame, 
        test_df_negatives: pd.DataFrame, 
        random_seed: int = 93,
        negatives_per_positive = (1), # it will work at higher values too, but more data will be slower
    ) -> pd.DataFrame:
    # sample negatives and combine with positives
    train_df = pd.concat(
        [
            train_df,
            train_df_negatives.sample(
                n=len(train_df) * negatives_per_positive, random_state=random_seed
            ),
        ]
    )
    test_df = pd.concat(
        [
            test_df,
            test_df_negatives.sample(
                n=len(test_df) * negatives_per_positive, random_state=random_seed
            ),
        ]
    )

    big_df = pd.concat([train_df, test_df])

    # sample it to make this faster
    big_df = big_df.sample(frac=0.3, random_state=random_seed).reset_index(drop=True)
    
    return big_df

In [ ]:
@pipeline
def load_and_process_data():
    train_df, test_df, _ = download_data()
    processed_train_df = process_input_data(train_df, dataset_type="train")
    processed_test_df = process_input_data(test_df, dataset_type="test")

    train_df_negatives = dataframe_of_negatives(processed_train_df, dataset_type="train")
    test_df_negatives = dataframe_of_negatives(processed_test_df, dataset_type="test")

    big_df = concatenate_dataframes(
        processed_train_df,
        train_df_negatives, 
        processed_test_df, 
        test_df_negatives, 
    )
    
load_and_process_data()

In [ ]:
# load data
from zenml.client import Client 

client = Client()

latest_run = client.get_pipeline("load_and_process_data").runs[-1]
big_df = latest_run.steps["concatenate_dataframes"].output.load()
big_df.head()

## 4. Calculate embeddings and cosine similarities

Here, I create a cache to save the embeddings. This is handy so that you don't have to pay again if you want to run the code again.

In [ ]:
from utils.embeddings_utils import get_embedding, cosine_similarity  # for embeddings

# establish a cache of embeddings to avoid recomputing
# cache is a dict of tuples (text, engine) -> embedding
def get_embedding_cache(embedding_cache_path: str) -> dict:
    try:
        with open(embedding_cache_path, "rb") as f:
            embedding_cache = pickle.load(f)
            logger.info("Loaded embeddings from cache.")
    except FileNotFoundError:
        precomputed_embedding_cache_path = "https://cdn.openai.com/API/examples/data/snli_embedding_cache.pkl"
        embedding_cache = pd.read_pickle(precomputed_embedding_cache_path)
        logger.info("Loaded embeddings from OpenAI.")
    return embedding_cache

embedding_cache_path = "data/get_embedding_cache.pkl"
embedding_cache = get_embedding_cache(embedding_cache_path)

# this function will get embeddings from the cache and save them there afterward
@step
def calculate_embeddings_and_cos_similarity(
    big_df: pd.DataFrame,
    default_embedding_engine: str = "babbage-similarity"
) -> pd.DataFrame:
    """Calculate embeddings and cosine similarity for each pair of texts in the dataframe."""
    
    # Set OpenAI key
    secret = Client().get_secret("openai")
    os.environ["OPENAI_API_KEY"] = secret.secret_values["api_key"]

    def get_embedding_with_cache(
        text: str,
        engine: str,
        embedding_cache_path: str,
    ) -> list:
        if (text, engine) not in embedding_cache.keys():
            # if not in cache, call API to get embedding
            logger.info(text)
            embedding_cache[(text, engine)] = get_embedding(text, engine)
            # save embeddings cache to disk after each update
            with open(embedding_cache_path, "wb") as embedding_cache_file:
                pickle.dump(embedding_cache, embedding_cache_file)
        return embedding_cache[(text, engine)]

    
    # create column of embeddings
    big_df = big_df.dropna(subset=['text_1', 'text_2'])
    for column in ["text_1", "text_2"]:
        big_df[f"{column}_embedding"] = big_df[column].apply(
            lambda text: get_embedding_with_cache(text, embedding_cache_path=embedding_cache_path, engine=default_embedding_engine)
        )

    # create column of cosine similarity between embeddings
    big_df["cosine_similarity"] = big_df.apply(
        lambda row: cosine_similarity(row["text_1_embedding"], row["text_2_embedding"]),
        axis=1,
    )
    
    
    # Return the big_df with embeddings and cosine similarity and the final embedding cache
    return big_df


In [ ]:
@pipeline
def load_and_process_data():
    train_df, test_df, _ = download_data()
    processed_train_df = process_input_data(train_df, dataset_type="train")
    processed_test_df = process_input_data(test_df, dataset_type="test")

    train_df_negatives = dataframe_of_negatives(processed_train_df, dataset_type="train")
    test_df_negatives = dataframe_of_negatives(processed_test_df, dataset_type="test")

    big_df = concatenate_dataframes(
        train_df,
        train_df_negatives, 
        test_df, 
        test_df_negatives, 
    )
    
    big_df_with_cos = calculate_embeddings_and_cos_similarity(big_df)
    
load_and_process_data()

In [ ]:
# load data
from zenml.client import Client 

client = Client()

latest_run = client.get_pipeline("load_and_process_data").runs[-1]
df = latest_run.steps["calculate_embeddings_and_cos_similarity"].output.load()
df

## 6. Plot distribution of cosine similarity

Here we measure similarity of text using cosine similarity. In our experience, most distance functions (L1, L2, cosine similarity) all work about the same. Note that our embeddings are already normalized to length 1, so cosine similarity is equivalent to dot product.

The graphs show how much the overlap there is between the distribution of cosine similarities for similar and dissimilar pairs. If there is a high amount of overlap, that means there are some dissimilar pairs with greater cosine similarity than some similar pairs.

The accuracy I compute is the accuracy of a simple rule that predicts 'similar (1)' if the cosine similarity is above some threshold X and otherwise predicts 'dissimilar (0)'.

In [ ]:
# calculate accuracy (and its standard error) of predicting label=1 if similarity>x
# x is optimized by sweeping from -1 to 1 in steps of 0.01
def accuracy_and_se(cosine_similarity: float, labeled_similarity: int) -> Tuple[float]:
    accuracies = []
    for threshold_thousandths in range(-1000, 1000, 1):
        threshold = threshold_thousandths / 1000
        total = 0
        correct = 0
        for cs, ls in zip(cosine_similarity, labeled_similarity):
            total += 1
            if cs > threshold:
                prediction = 1
            else:
                prediction = -1
            if prediction == ls:
                correct += 1
        accuracy = correct / total
        accuracies.append(accuracy)
    a = max(accuracies)
    n = len(cosine_similarity)
    standard_error = (a * (1 - a) / n) ** 0.5  # standard error of binomial
    return a, standard_error


# check that training and test sets are balanced
px.histogram(
    df,
    x="cosine_similarity",
    color="label",
    barmode="overlay",
    width=500,
    facet_row="dataset",
).show()

for dataset in ["train", "test"]:
    data = df[df["dataset"] == dataset]
    a, se = accuracy_and_se(data["cosine_similarity"], data["label"])
    print(f"{dataset} accuracy: {a:0.1%} ± {1.96 * se:0.1%}")


## 7. Optimize the matrix using the training data provided

In [ ]:
def embedding_multiplied_by_matrix(
    embedding: List[float], matrix: torch.tensor
) -> np.array:
    embedding_tensor = torch.tensor(embedding).float()
    modified_embedding = embedding_tensor @ matrix
    modified_embedding = modified_embedding.detach().numpy()
    return modified_embedding


# compute custom embeddings and new cosine similarities
@step
def apply_matrix_to_embeddings_dataframe(matrix: torch.tensor, df: pd.DataFrame) -> pd.DataFrame:
    for column in ["text_1_embedding", "text_2_embedding"]:
        df[f"{column}_custom"] = df[column].apply(
            lambda x: embedding_multiplied_by_matrix(x, matrix)
        )
    df["cosine_similarity_custom"] = df.apply(
        lambda row: cosine_similarity(
            row["text_1_embedding_custom"], row["text_2_embedding_custom"]
        ),
        axis=1,
    )
    return df


In [ ]:
def optimize_matrix(
    df: pd.DataFrame,
    modified_embedding_length: int = 2048,  # in my brief experimentation, bigger was better (2048 is length of babbage encoding)
    batch_size: int = 100,
    max_epochs: int = 100,
    learning_rate: float = 100.0,  # seemed to work best when similar to batch size - feel free to try a range of values
    dropout_fraction: float = 0.0,  # in my testing, dropout helped by a couple percentage points (definitely not necessary)
    print_progress: bool = True,
    save_results: bool = True,
) -> torch.tensor:
    """Return matrix optimized to minimize loss on training data."""
    run_id = random.randint(0, 2 ** 31 - 1)  # (range is arbitrary)
    # convert from dataframe to torch tensors
    # e is for embedding, s for similarity label
    def tensors_from_dataframe(
        df: pd.DataFrame,
        embedding_column_1: str,
        embedding_column_2: str,
        similarity_label_column: str,
    ) -> Tuple[torch.tensor]:
        e1 = np.stack(np.array(df[embedding_column_1].values))
        e2 = np.stack(np.array(df[embedding_column_2].values))
        s = np.stack(np.array(df[similarity_label_column].astype("float").values))

        e1 = torch.from_numpy(e1).float()
        e2 = torch.from_numpy(e2).float()
        s = torch.from_numpy(s).float()

        return e1, e2, s

    e1_train, e2_train, s_train = tensors_from_dataframe(
        df[df["dataset"] == "train"], "text_1_embedding", "text_2_embedding", "label"
    )
    e1_test, e2_test, s_test = tensors_from_dataframe(
        df[df["dataset"] == "test"], "text_1_embedding", "text_2_embedding", "label"
    )

    # create dataset and loader
    dataset = torch.utils.data.TensorDataset(e1_train, e2_train, s_train)
    train_loader = torch.utils.data.DataLoader(
        dataset, batch_size=batch_size, shuffle=True
    )

    # define model (similarity of projected embeddings)
    def model(embedding_1, embedding_2, matrix, dropout_fraction=dropout_fraction):
        e1 = torch.nn.functional.dropout(embedding_1, p=dropout_fraction)
        e2 = torch.nn.functional.dropout(embedding_2, p=dropout_fraction)
        modified_embedding_1 = e1 @ matrix  # @ is matrix multiplication
        modified_embedding_2 = e2 @ matrix
        similarity = torch.nn.functional.cosine_similarity(
            modified_embedding_1, modified_embedding_2
        )
        return similarity

    # define loss function to minimize
    def mse_loss(predictions, targets):
        difference = predictions - targets
        return torch.sum(difference * difference) / difference.numel()

    # initialize projection matrix
    embedding_length = len(df["text_1_embedding"].values[0])
    matrix = torch.randn(
        embedding_length, modified_embedding_length, requires_grad=True
    )

    epochs, types, losses, accuracies, matrices = [], [], [], [], []
    for epoch in range(1, 1 + max_epochs):
        # iterate through training dataloader
        for a, b, actual_similarity in train_loader:
            # generate prediction
            predicted_similarity = model(a, b, matrix)
            # get loss and perform backpropagation
            loss = mse_loss(predicted_similarity, actual_similarity)
            loss.backward()
            # update the weights
            with torch.no_grad():
                matrix -= matrix.grad * learning_rate
                # set gradients to zero
                matrix.grad.zero_()
        # calculate test loss
        test_predictions = model(e1_test, e2_test, matrix)
        test_loss = mse_loss(test_predictions, s_test)

        # compute custom embeddings and new cosine similarities
        apply_matrix_to_embeddings_dataframe(matrix, df)

        # calculate test accuracy
        for dataset in ["train", "test"]:
            data = df[df["dataset"] == dataset]
            a, se = accuracy_and_se(data["cosine_similarity_custom"], data["label"])

            # record results of each epoch
            epochs.append(epoch)
            types.append(dataset)
            losses.append(loss.item() if dataset == "train" else test_loss.item())
            accuracies.append(a)
            matrices.append(matrix.detach().numpy())

            # optionally print accuracies
            if print_progress is True:
                print(
                    f"Epoch {epoch}/{max_epochs}: {dataset} accuracy: {a:0.1%} ± {1.96 * se:0.1%}"
                )

    data = pd.DataFrame(
        {"epoch": epochs, "type": types, "loss": losses, "accuracy": accuracies}
    )
    data["run_id"] = run_id
    data["modified_embedding_length"] = modified_embedding_length
    data["batch_size"] = batch_size
    data["max_epochs"] = max_epochs
    data["learning_rate"] = learning_rate
    data["dropout_fraction"] = dropout_fraction
    data[
        "matrix"
    ] = matrices  # saving every single matrix can get big; feel free to delete/change
    if save_results is True:
        data.to_csv(f"{run_id}_optimization_results.csv", index=False)

    return data


@step
def tune_matrix(big_df: pd.DataFrame) -> pd.DataFrame:
    # example hyperparameter search
    # I recommend starting with max_epochs=10 while initially exploring
    results = []
    max_epochs = 30
    dropout_fraction = 0.2
    for batch_size, learning_rate in [(10, 10), (100, 100), (1000, 1000)]:
        result = optimize_matrix(
            df=big_df,
            batch_size=batch_size,
            learning_rate=learning_rate,
            max_epochs=max_epochs,
            dropout_fraction=dropout_fraction,
            save_results=False,
        )
        results.append(result)
        
    runs_df = pd.concat(results)
    return runs_df


In [ ]:
@pipeline
def load_and_process_data():
    train_df, test_df, _ = download_data()
    processed_train_df = process_input_data(train_df, dataset_type="train")
    processed_test_df = process_input_data(test_df, dataset_type="test")

    train_df_negatives = dataframe_of_negatives(processed_train_df, dataset_type="train")
    test_df_negatives = dataframe_of_negatives(processed_test_df, dataset_type="test")

    big_df = concatenate_dataframes(
        train_df,
        train_df_negatives, 
        test_df, 
        test_df_negatives, 
    )
    
    big_df_with_cos, embedding_cache = calculate_embeddings_and_cos_similarity(big_df)
    
    big_df_with_embeddings = apply_matrix_to_embeddings_dataframe(big_df_with_cos)
    
    tune_matrix(big_df_with_embeddings)
    
load_and_process_data()

In [ ]:
# load data
from zenml.client import Client 

client = Client()

latest_run = client.get_pipeline("load_and_process_data").runs[-1]
runs_df = latest_run.steps["tune_matrix"].output.load()
runs_df

In [ ]:


# plot training loss and test loss over time
px.line(
    runs_df,
    line_group="run_id",
    x="epoch",
    y="loss",
    color="type",
    hover_data=["batch_size", "learning_rate", "dropout_fraction"],
    facet_row="learning_rate",
    facet_col="batch_size",
    width=500,
).show()

# plot accuracy over time
px.line(
    runs_df,
    line_group="run_id",
    x="epoch",
    y="accuracy",
    color="type",
    hover_data=["batch_size", "learning_rate", "dropout_fraction"],
    facet_row="learning_rate",
    facet_col="batch_size",
    width=500,
).show()


## 8. Plot the before & after, showing the results of the best matrix found during training

The better the matrix is, the more cleanly it will separate the similar and dissimilar pairs.

In [ ]:
# apply result of best run to original data
best_run = runs_df.sort_values(by="accuracy", ascending=False).iloc[0]
best_matrix = best_run["matrix"]
apply_matrix_to_embeddings_dataframe.entrypoint(best_matrix, df)


In [ ]:
# plot similarity distribution BEFORE customization
px.histogram(
    df,
    x="cosine_similarity",
    color="label",
    barmode="overlay",
    width=500,
    facet_row="dataset",
).show()

test_df = df[df["dataset"] == "test"]
a, se = accuracy_and_se(test_df["cosine_similarity"], test_df["label"])
print(f"Test accuracy: {a:0.1%} ± {1.96 * se:0.1%}")

# plot similarity distribution AFTER customization
px.histogram(
    df,
    x="cosine_similarity_custom",
    color="label",
    barmode="overlay",
    width=500,
    facet_row="dataset",
).show()

a, se = accuracy_and_se(test_df["cosine_similarity_custom"], test_df["label"])
print(f"Test accuracy after customization: {a:0.1%} ± {1.96 * se:0.1%}")


In [ ]:
best_matrix  # this is what you can multiply your embeddings by
